In [ ]:
pwd

In [ ]:
%cd drive/MyDrive/X3D

In [ ]:
%cd fine-tune

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -r requirements.txt

In [ ]:
# 1. 기존 torchvision 제거
!pip uninstall -y torchvision

# 2. PyTorch와 같은 CUDA 11.8 버전용 torchvision 설치
!pip install --upgrade --force-reinstall torchvision --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# 파인튜닝
!python -u src/train.py

In [ ]:
# 에폭 전체 다 못 돌고 끝났다면, 따로 test 성능 측정
!python src/eval.py

### 추가로 성능 검증

In [ ]:
# train과 test 중복 확인
!comm -12 <(ls ../dataset_encoded/train/swoon | sort) <(ls ../dataset_encoded/test/swoon | sort)

In [ ]:
# 클래스 불균형 확인
import os
from collections import Counter

test_root = "../dataset_encoded/test"
class_counts = {cls: len(os.listdir(os.path.join(test_root, cls)))
                for cls in sorted(os.listdir(test_root))}
print(class_counts)

In [ ]:
import os
import yaml
import torch
import torch.nn.functional as F
from tqdm import tqdm
from collections import defaultdict

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
)

import matplotlib.pyplot as plt
from pytorch_lightning import seed_everything

from src.data_module import VideoDataModule
from src.model_module import X3DFineTuner

# 1. config 로드
with open("/content/drive/MyDrive/X3D/fine-tune/configs/default.yaml", "r") as f:
    CFG = yaml.safe_load(f)

seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 데이터 및 모델
dm = VideoDataModule(CFG)
dm.setup("test")
test_loader = dm.test_dataloader()

model = X3DFineTuner.load_from_checkpoint(
    checkpoint_path="/content/drive/MyDrive/X3D/fine-tune/checkpoints/x3d-epoch=26-val_loss=0.02.ckpt",
    cfg=CFG,
)
model.eval().to(device)

# 3. 클래스 추출
class_names = sorted(os.listdir(os.path.join(CFG["data_path"], "test")))
num_classes = len(class_names)
print(f"클래스 ({num_classes}):", class_names)

# 4. 예측 및 정답 수집
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        video = batch["video"].to(device)
        label = batch["label"].to(device)

        logits = model(video)
        preds = torch.argmax(F.softmax(logits, dim=1), dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(label.cpu().tolist())

# 5. 혼동행렬 출력
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap="Blues", xticks_rotation=45)
plt.title("Confusion Matrix (Test Set)")
plt.grid(False)
plt.show()

# 6. classification report (macro/micro/weighted 포함)
report = classification_report(
    all_labels, all_preds, target_names=class_names, digits=4
)
print("\nClassification Report:")
print(report)